In [14]:
import re

In [1]:
import pandas as pd
import numpy as np
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

#vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(2,4), min_df=5, sublinear_tf=True)
sentence_model = SentenceTransformer("paraphrase-MiniLM-L6-v2")
topic_model = BERTopic(embedding_model=sentence_model)#, vectorizer_model=vectorizer)

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/314 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [22]:
text_df = pd.read_csv('tweets.csv')

In [23]:
def clean_tweets(df, tweets):
    """
    Clean text column
    df = dataframe
    tweets (string) = column name containing tweets
    """
    # lowercase text
    df[tweets] = df[tweets].str.lower()

    # remove URLs
    df[tweets] = df[tweets].map(lambda x: re.sub('http[s]?:\/\/[^\s]*', ' ', x))

    # remove URL cutoffs
    df[tweets] = df[tweets].map(lambda x: re.sub('\\[^\s]*', ' ', x))

    # remove spaces
    df[tweets] = df[tweets].map(lambda x: re.sub('\n', ' ', x))

    # remove picture URLs
    df[tweets] = df[tweets].map(lambda x: re.sub('pic.twitter.com\/[^\s]*', ' ', x))

    # remove blog/map type
    df[tweets] = df[tweets].map(lambda x: re.sub('blog\/maps\/info\/[^\s]*', ' ', x))

    # remove hashtags =
    df[tweets] = df[tweets].map(lambda x: re.sub("\#[\w]*", "", x))

    # remove and signs
    df[tweets] = df[tweets].map(lambda x: re.sub("\&amp;", "", x))

    # remove AT users
    df[tweets] = df[tweets].map(lambda x: re.sub("\@[\w]*", "", x))

    # remove single quotations
    df[tweets] = df[tweets].map(lambda x: re.sub("'", "", x))
    df[tweets] = df[tweets].map(lambda x: re.sub("'", "", x))

    # remove characters that are not word characters or digits
    df[tweets] = df[tweets].map(lambda x: re.sub("[^\w\d]", " ", x))

    # remove all characters that are not letters
    #df[tweets] = df[tweets].map(lambda x: re.sub("[^a-zA-Z]", " ", x))

    # remove multiple spaces
    df[tweets] = df[tweets].map(lambda x: re.sub("\s{2,6}", " ", x))

    # drop duplicate rows
    #df.drop_duplicates(subset='text', keep='first', inplace=True)

    # remove multiple spaces
    df[tweets] = df[tweets].map(lambda x: re.sub("\s{3,20}", "", x))

    return df

In [24]:
text_df = clean_tweets(text_df, 'Tweet')

In [25]:
text_df.head()

,ID,Foundation_Name,Tweet,theme_1,theme_2,Tweet_Date,Tweeted_by_Foundation,Query_Term,Handle_Type,Region
0,1500694853277335552,Anfrel,here are the incidents and developments in th...,incident,NaN,2022-03-07,True,@Anfrel,Org,Asia
1,1501202326798974976,WITNESS_Asia,rt the rohingya are the most persecuted nation...,rohingya,NaN,2022-03-08,True,@WITNESS_Asia,Org,Asia
2,1501026064281309184,Altsean,rt more from here is the fantastic nang moet m...,women,NaN,2022-03-08,True,@Altsean,Org,Myanmar
3,1501022642299670537,Anfrel,as we celebrate and today march 8 the internat...,women,NaN,2022-03-08,True,@Anfrel,Org,Asia
4,1501261106957934600,ChinHumanRights,today issued a new report detailing human rig...,report,chin,2022-03-08,True,@ChinHumanRights,Org,Myanmar


In [27]:
text_df['tweet_pp'] = text_df['Tweet'].apply(lambda t: str(t))
text_df['tweet_pp'] = text_df['tweet_pp'].apply(lambda t: t.replace('rt : ', ''))
text_df['tweet_pp'] = text_df['tweet_pp'].apply(lambda t: t.replace('rt', ''))

#text_df.fillna('N/A')
print(len(text_df))
text_df = text_df[text_df['tweet_pp'] != 'N/A']
print(len(text_df))
text_df.head()

131
131


,ID,Foundation_Name,Tweet,theme_1,theme_2,Tweet_Date,Tweeted_by_Foundation,Query_Term,Handle_Type,Region,tweet_pp
0,1500694853277335552,Anfrel,here are the incidents and developments in th...,incident,NaN,2022-03-07,True,@Anfrel,Org,Asia,here are the incidents and developments in th...
1,1501202326798974976,WITNESS_Asia,rt the rohingya are the most persecuted nation...,rohingya,NaN,2022-03-08,True,@WITNESS_Asia,Org,Asia,the rohingya are the most persecuted nation r...
2,1501026064281309184,Altsean,rt more from here is the fantastic nang moet m...,women,NaN,2022-03-08,True,@Altsean,Org,Myanmar,more from here is the fantastic nang moet moe...
3,1501022642299670537,Anfrel,as we celebrate and today march 8 the internat...,women,NaN,2022-03-08,True,@Anfrel,Org,Asia,as we celebrate and today march 8 the internat...
4,1501261106957934600,ChinHumanRights,today issued a new report detailing human rig...,report,chin,2022-03-08,True,@ChinHumanRights,Org,Myanmar,today issued a new repo detailing human right...


In [28]:
topics, _ = topic_model.fit_transform(text_df['tweet_pp'].values)

In [51]:
topic_model.get_topic(2)


[('the', 0.10534230640485337),
 ('genocide', 0.09984253935385833),
 ('determination', 0.0843676091855392),
 ('rohingya', 0.0746171519943874),
 ('to', 0.06633275447582801),
 ('in', 0.06041264057522896),
 ('of', 0.05927878095001347),
 ('an', 0.05847051451836827),
 ('for', 0.05627444197043694),
 ('is', 0.056171369274846054)]

In [36]:
topic_grams = []
for k in range(len(set(topics))):
    cur_top = topic_model.get_topic(k)
    if cur_top:
        cur_d = {'topic number': k}
        for j in range(10):
            cur_d[f'topic ngram {j+1}'] = cur_top[j][0]
        topic_grams.append(cur_d)
topics_df = pd.DataFrame(topic_grams)

In [37]:
topics_df.head()

,topic number,topic ngram 1,topic ngram 2,topic ngram 3,topic ngram 4,topic ngram 5,topic ngram 6,topic ngram 7,topic ngram 8,topic ngram 9,topic ngram 10
0,0,myanmar,of,the,in,political,prisoners,military,for,new,by
1,1,the,here,and,2022,of,to,are,from,in,march
2,2,the,genocide,determination,rohingya,to,in,of,an,for,is
3,3,and,to,the,women,of,join,our,you,thank,that


In [40]:
for k in np.where(np.array(topics)==0)[0][:15]:
    print(k, text_df['Foundation_Name'].values[k])
    print(text_df['tweet_pp'].values[k])
    print('='*100)

7 waiwainu
 new event 8th march 8pm myanmar time presents break the bias reject dictatorship an event for internat 
18 PVamplify
 an impoant briefing document from on why the un must establish a jurisdiction for grave crimes in myanmar th 
20 Altsean
 this repo is absolutely damming of many investment funds that are marketed as more responsible yet they invest in many f 
21 JusticeMyanmar
 myanmar local international campaigners raise the alarm of the potential sale of norway to the mili 
22 JusticeMyanmar
 esg funds scolded for investing in myanmar arms suppliers via 
25 Altsean
 there is nothing socially responsible about investing in arms manufacturers that sustain militarys attacks 
31 waiwainu
 mon avenues to accountability for sexual violence in myanmar 
33 PVamplify
 so called opposing sides in myanmar these are the military and the people the military will never allow agencies to 
41 WomensPeaceNW
 celebrated on 8 march saw myanmar s women speak truth to power calls from organi

In [41]:
text_df['Topic_Number'] = topics


In [46]:
text_df['Topic_Number'].unique()

array([ 1, -1,  3,  0,  2])

In [48]:
text_df[text_df['Topic_Number'] == 3]

,ID,Foundation_Name,Tweet,theme_1,theme_2,Tweet_Date,Tweeted_by_Foundation,Query_Term,Handle_Type,Region,tweet_pp,Topic_Number
3,1501022642299670537,Anfrel,as we celebrate and today march 8 the internat...,women,NaN,2022-03-08,True,@Anfrel,Org,Asia,as we celebrate and today march 8 the internat...,3
9,1500996698004496391,waiwainu,rt i thought giving the women s voice and unde...,women,NaN,2022-03-08,True,@waiwainu,Activist,Myanmar,i thought giving the women s voice and unders...,3
23,1501975910282674177,waiwainu,rt monday join us for a urgent side event duri...,event,women,2022-03-10,True,@waiwainu,Activist,Myanmar,monday join us for a urgent side event during...,3
24,1502159331701125121,OCHAAsiaPac,rt they re part of first responders our facili...,NaN,NaN,2022-03-11,True,@OCHAAsiaPac,Org,Asia Pacific,they re pa of first responders our facilitato...,3
59,1505739093837168641,PVamplify,rt join and sac ms marzuki darusman on tuesday...,policy,NaN,2022-03-21,True,@PVamplify,Org,Myanmar,join and sac ms marzuki darusman on tuesday t...,3
65,1506336297648013315,WomensPeaceNW,rt thank you and for your wonderful statement ...,NaN,NaN,2022-03-22,True,@WomensPeaceNW,Org,Myanmar,thank you and for your wonderful statement it...,3
67,1506103622437941254,Anfrel,congratulates the people of for a peaceful ge...,elections,NaN,2022-03-22,True,@Anfrel,Org,Asia,congratulates the people of for a peaceful ge...,3
68,1506390371517018121,YasminJUllah,rt such a joy to finally meet so many other a...,event,NaN,2022-03-22,True,@YasminJUllah,Activist,Myanmar,such a joy to finally meet so many other ama...,3
70,1506087963045908481,YasminJUllah,rt so great to be with these amazing women tod...,event,women,2022-03-22,True,@YasminJUllah,Activist,Myanmar,so great to be with these amazing women today...,3
76,1506116503443288065,waiwainu,rt please watch honor to join friends and to t...,event,NaN,2022-03-22,True,@waiwainu,Activist,Myanmar,please watch honor to join friends and to tal...,3


In [50]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,35,-1_the_of_in_to
1,0,31,0_myanmar_of_the_in
2,1,27,1_the_here_and_2022
3,2,24,2_the_genocide_determination_rohingya
4,3,14,3_and_to_the_women
